以 ResNet50 网络模型为例，一般将 ResNet50 去除最后一层后的网络作为新任务的特
征提取子网络，即利用在 ImageNet 数据集上预训练好的网络参数初始化，并根据自定义任
务的类别追加一个对应数据类别数的全连接分类层或子网络，从而可以在预训练网络的基
础上快速、高效地学习新任务

In [ ]:
from tensorflow import keras
import tensorflow as tf
# 加载 ImageNet 预训练网络模型，并去掉最后一层
resnet = keras.applications.ResNet50(weights='imagenet',include_top=False)
resnet.summary()
# 测试网络的输出
x = tf.random.normal([4,224,224,3])
out = resnet(x) # 获得子网络的输出
out.shape



上述代码自动从服务器下载模型结构和在 ImageNet 数据集上预训练好的网络参数。通过设
置 include_top 参数为 False，可以选择去掉 ResNet50 最后一层，此时网络的输出特征图大
小为[𝑏, 7,7,2048]。对于某个具体的任务，需要设置自定义的输出节点数，以 100 类的分类
任务为例，我们在 ResNet50 基础上重新构建新网络。新建一个池化层(这里的池化层暂时
可以理解为高、宽维度下采样的功能)，将特征从[𝑏, 7,7,2048]降维到[𝑏, 2048]。

In [ ]:
from tensorflow.keras import layers

# 新建池化层
global_average_layer = layers.GlobalAveragePooling2D()
# 利用上一层的输出作为本层的输入，测试其输出
x = tf.random.normal([4,7,7,2048])

In [ ]:
# 池化层降维，形状由[4,7,7,2048]变为[4,1,1,2048],删减维度后变为[4,2048]
out = global_average_layer(x)
print(out.shape)

In [ ]:
# 新建全连接层
fc = layers.Dense(100)
# 利用上一层的输出[4,2048]作为本层的输入，测试其输出
x = tf.random.normal([4,2048])
out = fc(x) # 输出层的输出为样本属于 100 类别的概率分布
print(out.shape)

在创建预训练的 ResNet50 特征子网络、新建的池化层和全连接层后，我们重新利用
Sequential 容器封装成一个新的网络：

In [ ]:
from tensorflow.keras.models import Sequential

# 重新包裹成我们的网络模型
mynet = Sequential([resnet, global_average_layer, fc])
mynet.summary()